We're ingesting files from https://github.com/rbhughes/purr_petra, which generates well-centric json extracted from DBISAM on Windows. Some datatypes are relatively flat; others will need quite a bit of processing to get them into a reasonable tabular format. We'll also generate a unique id based on the source project/repo + datatype + whatever else.

databricks cli can't do wildcards on cp, so loop to copy into the volume instead:

`bryan@ichabod mac_bucket % for file in *_formation.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/formation/
done`

In [0]:
df = spark.read.format('json').load("/Volumes/geodata/petra/well/")



In [0]:
# df.printSchema()
# df.dtypes
display(df)

In [0]:
# from pyspark.sql.functions import to_timestamp, col
# from pyspark.sql import DataFrame

# def convert_iso_to_timestamp(df: DataFrame, string_col: str, new_col: str) -> DataFrame:
#     return df.withColumn(new_col, to_timestamp(col(string_col), "yyyy-MM-dd'T'HH:mm:ss"))

import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def convert_iso_to_timestamp(df: DataFrame, string_col: str, new_col: str) -> DataFrame:
    return df.withColumn(new_col, F.to_timestamp(F.col(string_col), "yyyy-MM-dd'T'HH:mm:ss"))

In [0]:
#df_with_coords = df.withColumn("locat_lat", df["locat.lat"])\
#    .withColumn("locat_lon", df["locat.lon"])\
#    .drop("locat")

display(df)

df_a = df.select("*",
    df.uwi.uwi.alias("uwi"),
    df.uwi.wsn.alias("wsn"),
    df.locat.lat.alias("surface_latitude"),
    df.locat.lon.alias("surface_longitude"),
    df.bhloc.lat.alias("bottom_latitude"),
    df.bhloc.lon.alias("bottom_longitude"),
    df.well.county.alias("county"),
    df.well.state.alias("state"),
    df.well.fieldname.alias("field_name"),
    df.well.histoper.alias("historical_operator"),
    df.well.label.alias("well_label"),
    df.well.leasename.alias("lease_name"),
    df.well.leasenumber.alias("lease_number"),
    df.well.operator.alias("operator"),
    df.well.prodfm.alias("producing_formation"),
    df.well.remarks.alias("remarks"),
    df.well.shortname.alias("short_name"),
    df.well.wellname.alias("well_name"),
    df.well.symcode.alias("symbol"),
    df.zdata.aband_date.alias("abandonment_date"),
    df.zflddef.active_datum.alias("active_datum"),
    df.zdata.active_datum_value.alias("active_datum_value"),
    df.zdata.comp_date.alias("completion_date"),
    df.zdata.cumgas.alias("cum_gas"),
    df.zdata.cumoil.alias("cum_oil"),
    df.zdata.cumwtr.alias("cum_water"),
    df.zdata.elev_df.alias("elev_df"),
    df.zdata.elev_gr.alias("elev_gr"),
    df.zdata.elev_kb.alias("elev_kb"),
    df.zdata.last_act_date.alias("last_activity_date"),
    df.zdata.permit_date.alias("permit_date"),
    df.zdata.rig_date.alias("rig_date"),
    df.zdata.report_date.alias("report_date"),
    df.zdata.spud_date.alias("spud_date"),
    df.zdata.td.alias('total_depth'),
    df.zdata.whipstock.alias("whipstock"),
    df.zdata.wtrdepth.alias("water_depth"),
    df.well.adddate.alias("app_row_created"),
    df.well.chgdate.alias("app_row_changed")

).drop("locat", "bhloc", "uwi", "well", "zdata", "zflddef")

display(df_a)

date_columns = [
    "abandonment_date", 
    "completion_date", 
    "last_activity_date", 
    "permit_date", 
    "report_date", 
    "spud_date",
    "app_row_created",
    "app_row_changed"
]

df_b = df_a
for col_name in date_columns:
    df_b = convert_iso_to_timestamp(df_b, col_name, col_name)

display(df_b)